# Segmenting and Clustering Neighborhoods in Toronto

## Week 3 Part I 

### Import libraries

In [8]:
! pip install folium==0.5.0
! pip install geopy
! pip install bs4

     |████████████████████████████████| 79 kB 8.2 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=6683ce6f3b9fff3fdfb20afcb49ba6dcdec228d19520134b5a002c2888015298
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=6ed29c6efa946e408c9263647780e66f782a1ec7f73c9d404865d8895b7968c4
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [9]:
import pandas as pd # library for data analsysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
print("Libraries imported.")

Libraries imported.


In [10]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

### Append the data into the 3 columns lists

In [11]:
# create 3 lists for the colums of the pandas dataframe
PostalCode = []
Borough = []
Neighborhood = []

for row in soup.find('table').find_all('tr'):  # find all the rows of the table
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCode.append(cells[0].text.rstrip('\n'))
        Borough.append(cells[1].text.rstrip('\n'))
        Neighborhood.append(cells[2].text.rstrip('\n'))

### Create a new DataFrame from the three lists

In [12]:
df = pd.DataFrame({"PostalCode": PostalCode, "Borough": Borough, "Neighborhood": Neighborhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignore cells with a borough that is Not assigned

In [13]:
# drop cells with a borough that is Not assigned
df2 = df[df.Borough != "Not assigned"].reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### group neighborhoods in the same borough

In [14]:
toronto = df2.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Neighborhood="Not assigned" same value as Borough

In [15]:
for index, row in toronto.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]        
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
toronto.shape

(103, 3)

### Loading Geospatal Coordinates CSV

In [17]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_241eaa55d6944ad0a1727cec221c3413 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='j3Vt4x2pvma42CubkNX7tlxhiCki8sijMXH2t-xDPrmA',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_241eaa55d6944ad0a1727cec221c3413.get_object(Bucket='capstoneprojectcoursera-donotdelete-pr-fscs4t6kybtjao',Key='Geospatial_Coordinates (1).csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geo_coordinates = pd.read_csv(body)
geo_coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# rename the column "PostalCode"
geo_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geo_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
toronto_geo_coordinates = toronto.assign(A = geo_coordinates.Latitude, B = geo_coordinates.Longitude) 
toronto_geo_coordinates.rename(columns={'A': 'Latitude', 'B': 'Longitude'}, inplace=True)
toronto_geo_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
